In [ ]:
# 양파 : 1201 # 배추 : 1001 # 상추 : 1005 # 사과 : 0601 # 무 : 1101 # 감자 : 0501 # 대파 : 1202 # 건고추 : 1207
# 마늘 : 1209 # 딸기 : 0804  # 방울토마토 : 0806 # 오이 : 0901 # 양배추 : 1004  # 고구마 : 0502  # 배 : 0602

item_nm = '무'
item_cd = 1101

In [ ]:
import pandas as pd
import glob
import os

# 기본 파일 경로
base_path = f"{item_nm}/유통공사_도매시장_{item_nm}.csv"

# retry 파일 목록 수집
retry_paths = glob.glob(f"{item_nm}/유통공사_retry_{item_nm}_*.csv")

# 기본 파일 읽기
df_list = [pd.read_csv(base_path, encoding="cp949", low_memory=False)]

# retry 파일들 읽기 (있을 경우에만)
for path in retry_paths:
    df_list.append(pd.read_csv(path, encoding="cp949", low_memory=False))

# 병합
df = pd.concat(df_list, ignore_index=True)

# 중복 제거 (전체 행 기준 또는 주요 열 기준)
df = df.drop_duplicates()

# plor_cd가 문자열이 아닐 가능성 대비
df['plor_cd'] = df['plor_cd'].fillna('').astype(str)

pattern = r'^[^0-9]+$'

condition = (
    df['totprc'].isna() | (df['totprc'] <= 0) |
    df['unit_tot_qty'].isna() | (df['unit_tot_qty'] <= 0) |
    df['plor_cd'].str.strip().isin(['0', '0.0']) |
    df['plor_cd'].str.match(pattern, na=False) |
    df['plor_nm'].isna() |
    (df['plor_nm'] == 0)
)

# 조건에 해당하는 행 추출
df_filtered = df[~condition]

# 결과 저장
df_filtered.to_csv(f"{item_nm}/유통공사_도매시장_{item_nm}_결측치제거.csv", index=False, encoding="cp949")

In [ ]:
import pandas as pd

# 1. 파일 경로 설정
input_path = f"{item_nm}/유통공사_도매시장_{item_nm}_결측치제거.csv"
output_path = f"{item_nm}/유통공사_도매시장_{item_nm}_한글컬럼명.csv"

# 2. CSV 전체를 문자열로 불러오기 (최초 경고 방지)
df = pd.read_csv(input_path, encoding='cp949', dtype=str)

# 3. 한글 컬럼명 설정
new_columns = [
    '평균가격(원)', '법인코드', '법인이름', '상품 대분류 코드', '상품 대분류 이름',
    '상품 중분류 코드', '상품 중분류 이름', '상품 소분류 코드', '상품 소분류 이름',
    '등급코드', '등급이름', '최고가(원)', '최저가(원)', '포장코드', '포장이름',
    '산지코드', '산지이름', '크기코드', '크기이름', '총가격(원)', '연월일',
    '매매구분', '단위코드', '단위(kg)', '단위물량(kg)', '단위총물량(kg)', '도매시장코드', '도매시장이름'
]

if len(df.columns) != len(new_columns):
    raise ValueError("컬럼 수가 일치하지 않습니다.")

df.columns = new_columns

# 4. 날짜 컬럼 변환
df['연월일'] = pd.to_datetime(df['연월일'], errors='coerce')

# 5. 숫자 컬럼 변환
numeric_columns = ['평균가격(원)', '최고가(원)', '최저가(원)', '총가격(원)', '단위물량(kg)', '단위총물량(kg)']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 6. 혼합 타입 경고 컬럼 처리: NaN을 빈 문자열로, 모두 문자열화
for col in ['포장이름', '산지이름', '크기이름']:
    df[col] = df[col].fillna('').astype(str)

# 7. CSV 저장 (완전 정리된 상태)
df.to_csv(output_path, index=False, encoding='cp949')

In [ ]:
import pandas as pd

# 파일 경로
input_path = f"{item_nm}/유통공사_도매시장_{item_nm}_한글컬럼명.csv"
output_path = f"{item_nm}/유통공사_{item_nm}_요약데이터.csv"

# CSV 파일 불러오기
df = pd.read_csv(input_path, encoding='cp949')

selected_columns = ['연월일', '상품 중분류 코드', '상품 중분류 이름', '상품 소분류 코드', '상품 소분류 이름',
                    '총가격(원)', '단위총물량(kg)', '산지코드', '산지이름']
df_selected = df[selected_columns]
df_selected.rename(columns={
    '상품 중분류 코드':'품목코드',
    '상품 중분류 이름':'품목명',
    '상품 소분류 코드':'품종코드',
    '상품 소분류 이름':'품종명',
    '총가격(원)':'총금액(원)',
    '단위총물량(kg)':'총거래량(kg)'
}, inplace=True)

df_selected['평균단가(원)'] = df_selected['총금액(원)'] / df_selected['총거래량(kg)']
df_selected['평균단가(원)'] = df_selected['평균단가(원)'].replace([float('inf'), -float('inf')], None)
df_selected.insert(df_selected.columns.get_loc('총거래량(kg)') + 1, '평균단가(원)', df_selected.pop('평균단가(원)'))

# 새 CSV로 저장
df_selected.to_csv(output_path, index=False, encoding='cp949')

In [ ]:
df = pd.read_csv(f"{item_nm}/유통공사_{item_nm}_요약데이터.csv", encoding = 'cp949')

df.isna().sum()

In [ ]:
import pandas as pd
import re

# CSV 파일 로드
df = pd.read_csv(f"{item_nm}/유통공사_{item_nm}_요약데이터.csv", encoding='cp949')

# 상품 중분류 이름 결측치 처리
df['품목코드'] = df['품목코드'].replace('', pd.NA).fillna(item_nm)
df['품목명'] = df['품목명'].replace('', pd.NA).fillna(item_nm)
df['품종코드'] = df['품종코드'].replace('', pd.NA).fillna(item_nm)
df['품종명'] = df['품종명'].replace('', pd.NA).fillna(item_nm)

# 산지코드 정제 함수 (문자 → 0, 앞자리 0 제거, 6자리 맞춤)
def clean_origin_code_backpad(code):
    code_str = str(code).strip()
    replaced = re.sub(r'\D', '0', code_str)      # 문자 → 0
    stripped = replaced.lstrip('0')              # 앞쪽 0 제거
    if not stripped:  # 모두 0이거나 제거 결과 없음
        stripped = '0'
    return stripped[:6].ljust(6, '0')            # 앞 6자리 + 뒤 0 패딩

# 적용
df['산지코드'] = df['산지코드'].apply(clean_origin_code_backpad)

# 저장
output_path = f"{item_nm}/유통공사_{item_nm}_요약데이터_정제.csv"
df.to_csv(output_path, index=False, encoding='cp949')

print(f"정제된 데이터가 저장되었습니다: {output_path}")

In [ ]:
df = pd.read_csv(f"{item_nm}/유통공사_{item_nm}_요약데이터_정제.csv", encoding = 'cp949')

df.isna().sum()

In [ ]:
import pandas as pd

# 파일 경로 설정
file_path = f"{item_nm}/유통공사_{item_nm}_요약데이터_정제.csv"

# CSV 파일 읽기 (경고 방지를 위해 low_memory=False 사용)
df = pd.read_csv(file_path, encoding="cp949", low_memory=False)

# 산지코드가 문자열로 처리되도록 변환
df['산지코드'] = df['산지코드'].fillna('').astype(str)

# 필터링 조건: totprc 또는 unit_tot_qty가 NaN이거나 0인 경우
pattern = r'^[^0-9]+$'

condition = (
    (df['산지코드'] == '') |
    (df['산지코드'].str.strip() == "0") |
    (df['산지코드'].str.strip() == "0.0")
#     df['plor_cd'].str.match(pattern, na=False)) 
#     (df['plor_nm'].isna() | (df['plor_nm'] == 0))
)

# 조건에 해당하는 행 추출
df_filtered = df[condition]

# 결과 저장
df_filtered.to_csv(f"{item_nm}/결측치_{item_nm}.csv", index=False, encoding="cp949")

In [ ]:
import pandas as pd

# 1. 파일 경로 설정
summary_path = f"{item_nm}/유통공사_{item_nm}_요약데이터_정제.csv"
region_code_path = "표준코드/산지코드_직팜.csv"

# 2. CSV 불러오기
summary_df = pd.read_csv(summary_path, encoding='cp949')
region_df = pd.read_csv(region_code_path, encoding='cp949')

# 3. 산지코드 범위 파싱 함수
def parse_code_range(code_str):
    if "~" in code_str:
        start, end = code_str.split("~")
    else:
        start = end = code_str
    return int(start), int(end)

# 4. 산지코드 범위 숫자 컬럼 생성
region_df[['start_code', 'end_code']] = region_df['산지코드'].apply(
    lambda x: pd.Series(parse_code_range(x))
)

# 5. 산지코드 → 직팜산지코드/이름 매핑 테이블 확장
expanded_rows = []
for _, row in region_df.iterrows():
    for code in range(row['start_code'], row['end_code'] + 1):
        expanded_rows.append({
            '산지코드': code,
            '직팜산지코드': row['직팜산지코드'],
            '직팜산지이름': row['직팜산지이름']
        })
expanded_map_df = pd.DataFrame(expanded_rows)

# 6. 요약 데이터 산지코드 정수형으로 변환
summary_df['산지코드'] = pd.to_numeric(summary_df['산지코드'], errors='coerce').astype('Int64')
summary_df = summary_df[summary_df['산지코드'].notna()].copy()
summary_df['산지코드'] = summary_df['산지코드'].astype(int)

# 7. 직팜 컬럼 제거 후 병합
summary_df = summary_df.drop(columns=['직팜산지코드', '직팜산지이름'], errors='ignore')
summary_df = pd.merge(summary_df, expanded_map_df, on='산지코드', how='left')

# 8. 산지이름이 누락된 경우 직팜산지이름으로 보완
summary_df['산지이름'] = summary_df['산지이름'].fillna(summary_df['직팜산지이름'])


# 9. 결과 저장
summary_df.to_csv(f"{item_nm}/{item_nm}요약데이터_직팜산지정리.csv", index=False, encoding='cp949')

In [ ]:
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜산지정리.csv", encoding = 'cp949')

df.isna().sum()

In [ ]:
import pandas as pd

# 파일 경로와 인코딩으로 데이터 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜산지정리.csv", encoding='cp949')

# 결측치가 하나라도 있는 행만 추출
df_na = df[df.isna().any(axis=1)]

# 결과 저장
output_path = f"{item_nm}/{item_nm}_결측치포함행.csv"
df_na.to_csv(output_path, index=False, encoding='cp949')

print(f"결측치 포함 행 {len(df_na)}개를 '{output_path}'로 저장했습니다.")

In [ ]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜산지정리.csv", encoding='cp949')

# 제거할 컬럼 리스트
columns_to_drop = ['산지코드', '산지이름', '직팜산지이름']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# 새로 추가할 컬럼들 (초기값은 None)
new_columns = [
    '휴일여부', '명절지수', '작기정보'
]
for col in new_columns:
    df[col] = None

# 결과 저장
df.to_csv(f"{item_nm}/{item_nm}요약데이터_컬럼정리완료.csv", index=False, encoding='cp949')

In [ ]:
import pandas as pd

# 파일 경로
summary_path = f"{item_nm}/{item_nm}요약데이터_컬럼정리완료.csv"
mapping_path = "기상/산지코드_직팜_관측지점_매핑완료.csv"

# CSV 파일 불러오기
df_summary = pd.read_csv(summary_path, encoding='cp949')
df_mapping = pd.read_csv(mapping_path, encoding='cp949')

# 중복 제거
df_mapping = df_mapping.drop_duplicates(subset='직팜산지코드')

# '직팜산지코드' 기준으로 '관측지점' 컬럼 병합
df_merge = pd.merge(
    df_summary,
    df_mapping[['직팜산지코드', '관측지점']],
    on='직팜산지코드',
    how='left'
)

# '작기정보' 컬럼 오른쪽에 '관측지점' 컬럼 삽입
if '작기정보' in df_merge.columns:
    insert_idx = df_merge.columns.get_loc('작기정보') + 1  # '작기정보' 다음 위치
    관측지점_series = df_merge.pop('관측지점')
    df_merge.insert(insert_idx, '관측지점', 관측지점_series)

# 결과 저장
df_merge.to_csv(f"{item_nm}/{item_nm}요약데이터_관측지점추가.csv", index=False, encoding='cp949')

In [ ]:
import pandas as pd

# 파일 경로 설정
crop_path = f"{item_nm}/{item_nm}요약데이터_관측지점추가.csv"
weather_path = "기상/기상청_일기요소.csv"
output_path = f"{item_nm}/{item_nm}요약데이터_기상병합완료.csv"

# 데이터 불러오기
df_crop = pd.read_csv(crop_path, encoding='cp949')
df_weather = pd.read_csv(weather_path, encoding='cp949')

# 날짜 처리 → YYYYMMDD 문자열
df_crop['연월일'] = pd.to_datetime(df_crop['연월일'], errors='coerce')
df_crop['날짜'] = df_crop['연월일'].dt.strftime('%Y%m%d')

df_weather['TM'] = pd.to_datetime(df_weather['TM'], errors='coerce')
df_weather['날짜'] = df_weather['TM'].dt.strftime('%Y%m%d')

# 관측지점/STN 모두 int로 통일 후 str 변환
df_crop['관측지점'] = df_crop['관측지점'].astype('Int64').astype(str)
df_weather['STN'] = df_weather['STN'].astype('Int64').astype(str)

# 병합 키 생성
df_crop['merge_key'] = df_crop['날짜'] + '_' + df_crop['관측지점']
df_weather['merge_key'] = df_weather['날짜'] + '_' + df_weather['STN']

# 날씨 컬럼 매핑
weather_cols = {
    'TA_AVG': '일평균기온',
    'TA_MAX': '최고기온',
    'TA_MIN': '최저기온',
    'HM_AVG': '평균상대습도',
    'RN_DAY': '강수량(mm)',
    'RN_60M_MAX': '1시간최고강수량(mm)'
}

# 병합용 데이터 준비
df_weather_subset = df_weather[['merge_key'] + list(weather_cols.keys())].copy()
df_weather_subset.rename(columns=weather_cols, inplace=True)

# 병합 수행
df_merged = pd.merge(df_crop, df_weather_subset, on='merge_key', how='left')

# 정리: 병합키 및 중간 날짜 컬럼 제거
df_merged.drop(columns=['merge_key', '날짜'], inplace=True)

# <NA> → 빈 문자열, NaN → 빈 문자열 처리
df_merged.replace("<NA>", "", inplace=True)
df_merged.fillna("", inplace=True)

# 결과 저장
df_merged.to_csv(output_path, index=False, encoding='cp949')

In [ ]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상병합완료.csv", encoding='cp949')

# 제거할 컬럼 리스트
columns_to_drop = ['관측지점']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# 결과 저장
df.to_csv(f"{item_nm}/{item_nm}요약데이터_기상.csv", index=False, encoding='cp949')

In [ ]:
# import pandas as pd

# df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상.csv", encoding='cp949')

# df = df[~(df['직팜산지코드'] == 2000)]

# df.to_csv(f"{item_nm}/{item_nm}요약데이터_기상_수입산제거.csv", index=False, encoding='cp949')

In [ ]:
# import pandas as pd

# # CSV 파일 불러오기
# df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상.csv", encoding='cp949')
# df['연월일'] = pd.to_datetime(df['연월일'])

# # 이상치 제거 함수
# def remove_outliers_iqr(group, column):
#     if len(group) < 4:
#         return group  # 너무 적으면 제거하지 않음
    
#     q1 = group[column].quantile(0.25)
#     q3 = group[column].quantile(0.75)
#     iqr = q3 - q1
#     lower = q1 - 1.5 * iqr
#     upper = q3 + 1.5 * iqr
#     return group[(group[column] >= lower) & (group[column] <= upper)]

# # 일별로 그룹핑하여 이상치 제거
# daily_filtered = df.groupby('연월일', group_keys=False).apply(
#     remove_outliers_iqr, column='평균단가(원)'
# )

# # 날짜별 제거 개수 계산
# original_counts = df.groupby('연월일').size().rename('원래수')
# filtered_counts = daily_filtered.groupby('연월일').size().rename('제거후수')
# removal_stats = pd.concat([original_counts, filtered_counts], axis=1).fillna(0).astype(int)
# removal_stats['제거수'] = removal_stats['원래수'] - removal_stats['제거후수']


# # 총 제거 수 계산
# total_removed = removal_stats['제거수'].sum()

# # 출력
# print("총 제거된 이상치 수:", total_removed)
# print("\n날짜별 제거 수 :")
# print(removal_stats[['제거수']])

# # 결과 CSV 저장
# daily_filtered.to_csv(f"{item_nm}/{item_nm}_이상치제거_일별기준.csv", index=False, encoding='cp949')

In [ ]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상.csv", encoding='cp949')
df['연월일'] = pd.to_datetime(df['연월일'])

# 주차(월요일 시작) 컬럼 생성
df['주차시작'] = df['연월일'].apply(lambda d: d - pd.Timedelta(days=d.weekday()))
df['주차끝'] = df['주차시작'] + pd.Timedelta(days=6)
df['주차'] = df['주차시작'].dt.strftime('%Y-%m-%d') + '~' + df['주차끝'].dt.strftime('%Y-%m-%d')

# 국산/수입산 분리
df_korean = df[df['직팜산지코드'] != 2000].copy()
df_imported = df[df['직팜산지코드'] == 2000].copy()

# 이상치 제거 함수
def remove_outliers_iqr(group, column):
    if len(group) < 4:
        return group
    q1 = group[column].quantile(0.25)
    q3 = group[column].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return group[(group[column] >= lower) & (group[column] <= upper)]

# 국산에 대해서만 이상치 제거
filtered_korean = (
    df_korean.groupby('주차', group_keys=False)
    .apply(remove_outliers_iqr, column='평균단가(원)')
    .reset_index(drop=True)
)
df_imported = df_imported.reset_index(drop=True)

# 제거 개수 계산
original_counts = df_korean.groupby('주차').size().rename('원래수')
filtered_counts = filtered_korean.groupby('주차').size().rename('제거후수')
removal_stats = pd.concat([original_counts, filtered_counts], axis=1).fillna(0).astype(int)
removal_stats['제거수'] = removal_stats['원래수'] - removal_stats['제거후수']
total_removed = removal_stats['제거수'].sum()

# 출력
print(" 총 제거된 이상치 수:", total_removed)

# 국산(이상치 제거됨) + 수입산 결합
weekly_filtered = pd.concat([filtered_korean, df_imported], ignore_index=True)

# 컬럼 정리
weekly_filtered = weekly_filtered[['주차'] + [col for col in weekly_filtered.columns if col != '주차']]
weekly_filtered = weekly_filtered.drop(columns=['주차시작', '주차끝'], errors='ignore')

# 연월일 기준 정렬
weekly_filtered = weekly_filtered.sort_values(by='연월일').reset_index(drop=True)

# 결과 CSV 저장
weekly_filtered.to_csv(f"{item_nm}/{item_nm}_이상치제거_주간기준.csv", index=False, encoding='cp949')

In [ ]:
small_group_weeks = df_korean.groupby('주차').filter(lambda g: len(g) < 4)['주차'].unique()

# 2. '연월일' 단위로 제거된 날짜 판별
original_dates = set(df_korean['연월일'].unique())
filtered_dates = set(filtered_korean['연월일'].unique())
removed_dates = original_dates - filtered_dates  # 제거되어 아예 사라진 날짜

# 3. 해당 날짜가 어느 주차에 속했는지 매핑
removed_rows = df_korean[df_korean['연월일'].isin(removed_dates)]
removed_rows = removed_rows[['연월일', '주차']].drop_duplicates()

# 4. 조건 통합: "문제성 있는 날짜 및 주차"
problematic_dates = removed_rows['연월일'].tolist()
problematic_weeks = sorted(set(small_group_weeks).union(set(removed_rows['주차'])))

# 5. 결과 출력
print(" 연월일이 모두 제거된 날짜 수:", len(problematic_dates))
print(" 연월일이 모두 제거된 날짜 목록:", problematic_dates)
print(" len<4 또는 날짜가 제거된 주차 수:", len(problematic_weeks))
print(" 해당 주차 목록:", problematic_weeks)

In [ ]:
import pandas as pd
import numpy as np

# 이상치 제거 후 파일 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}_이상치제거_주간기준.csv", encoding='cp949')

# 국산만 기준으로 주간 평균 단가 계산 (df_korean 없이)
weekly_avg = df[df['직팜산지코드'] != 2000].groupby('주차')['평균단가(원)'].mean().rename('주간평균단가(원)')

# 병합
df = df.merge(weekly_avg, on='주차', how='left')

# 등급 부여
df['등급이름'] = np.select(
    condlist=[
        (df['직팜산지코드'] != 2000) & (df['평균단가(원)'] > df['주간평균단가(원)'] * 1.3),
        (df['직팜산지코드'] != 2000) & (df['평균단가(원)'] > df['주간평균단가(원)'] * 0.7) & (df['평균단가(원)'] <= df['주간평균단가(원)'] * 1.3),
        (df['직팜산지코드'] != 2000) & (df['평균단가(원)'] <= df['주간평균단가(원)'] * 0.7),
        (df['직팜산지코드'] == 2000)
    ],
    choicelist=['고', '중', '저', '수입산'],
    default='등급불명'
)

# 컬럼 정리
cols = list(df.columns)
for col in ['주간평균단가(원)', '등급이름']:
    if col in cols:
        cols.remove(col)

idx_price = cols.index('평균단가(원)') if '평균단가(원)' in cols else - 1
idx_total = cols.index('총금액(원)') if '총금액(원)' in cols else - 1

if idx_price != -1:
    cols.insert(idx_price + 1, '주간평균단가(원)')
if idx_total != -1:
    cols.insert(idx_total, '등급이름')

df = df[cols]

df.to_csv(f"{item_nm}/{item_nm}_이상치제거_주간기준_등급이름.csv", index=False, encoding='cp949')

In [ ]:
import pandas as pd

# 등급포함 CSV 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}_이상치제거_주간기준_등급이름.csv", encoding='cp949')

# 등급이름 → 등급코드 매핑
grade_map = {
    '고': 11,
    '중': 12,
    '저': 13,
    '수입산': 20,
    '등급불명': 99
}

# 등급코드 컬럼 생성
df['등급코드'] = df['등급이름'].map(grade_map)

# 등급이름 앞에 등급코드 삽입
cols = list(df.columns)
if '등급이름' in cols and '등급코드' in cols:
    cols.remove('등급코드')
    idx = cols.index('등급이름')
    cols.insert(idx, '등급코드')
    df = df[cols]

# 결과 저장
df.to_csv(f"{item_nm}/{item_nm}_이상치제거_주간기준_등급코드.csv", index=False, encoding='cp949')

In [ ]:
import pandas as pd

# 1. 두 파일 불러오기
df = pd.read_csv(f"EDA/{item_nm}(EDA용)_스케일링만.csv", encoding="cp949")
df_ext = pd.read_csv("표준코드/factor_external_weekly.csv", encoding="cp949")

columns_to_drop = ["holiday_flag", "holiday_score", "grow_score"]
df = df.drop(columns=columns_to_drop, errors="ignore")

# 2. factor_external_weekly에서 weekno → year, week 분리
if 'weekno' in df_ext.columns:
    df_ext['year'] = df_ext['weekno'].astype(str).str[:4].astype(int)
    df_ext['week'] = df_ext['weekno'].astype(str).str[4:].astype(int)
    df_ext.drop(columns=['weekno'], inplace=True)

# 3. item_code가 1101인 데이터만 필터링
df_ext_filtered = df_ext[df_ext['item_code'] == item_cd]
    
# 4. 중복 제거 (year, week 기준)
df_ext_unique = df_ext_filtered.drop_duplicates(subset=['year', 'week'])

# 5. 병합 (year, week 기준)
df_merged = pd.merge(
    df,
    df_ext_unique[['year', 'week', 'holiday_flag', 'holiday_score', 'grow_score']],
    on=['year', 'week'],
    how='left'
)

# 6. '평균단가(원)' 왼쪽에 삽입
price_col_index = df_merged.columns.get_loc('평균단가(원)')
for col in ['grow_score', 'holiday_score', 'holiday_flag']:
    if col in df_merged.columns:
        df_merged.insert(price_col_index, col, df_merged.pop(col))

# 7. 결과 저장
df_merged.to_csv(f"EDA/{item_nm}(EDA용)_스케일링만_병합.csv", index=False, encoding="cp949")